In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import Image

Funcion para obtener la informacion de la pagina 12

In [4]:
def obtener_info(s_notas):
    diccionario = {}
    #Extraemos la fecha
    fecha = s_notas.find('span',attrs={'pubdate':'pubdate'})
    if fecha:
        diccionario['Fecha'] = fecha.get('datetime')
    else:
        diccionario['Fecha'] = None
     #Extraemos el titulo
    titulo = s_notas.find('h1',attrs={'class':'article-title'})
    if titulo:
        diccionario['Titulo'] = titulo.text
    else:
        diccionario['Titulo'] = None
    #Extraemos la volanda
    volanda = s_notas.find('h2',attrs={'class':'article-prefix'})
    if volanda:
        diccionario['Volanda'] = volanda.text
    else:
        diccionario['Volanda'] = None
    #Extraemos el copete
    copete = s_notas.find('div',attrs={'class':'article-summary'})
    if copete:
        diccionario['Copete'] = copete.text
    else:
        diccionario['Copete'] = None
    #Extraemos el autor
    autor = s_notas.find('div',attrs={'class':'article-author'})
    if autor:
        diccionario['Autor'] = autor.text
    else:
        diccionario['Autor'] = None
    #Extraemos la imagen
    media = s_notas.find('div',attrs={'class':'article-main-media'})
    if media:
        imagenes = media.find_all('img')
        if len(imagenes)==0:
            print('no se encontro imagen')
        else:
            imagen = imagenes[-1]
            img_src = imagen.get('data-src')
            try:
                img_get = requests.get(img_src)
                if img_get.status_code == 200:
                    diccionario['Imagen'] = img_get.content
                else:
                    diccionario['Imagen'] = None
            except:
                print('No se encontro imagen')
    #Extremos el cuerpo:
    cuerpo  = s_notas.find('div',attrs={'class':'article-text'})
    if cuerpo:
        diccionario['Cuerpo de texto'] = cuerpo.text
    else:
        diccionario['Cuerpo de texto'] = None
    return diccionario

verificando que el reponse sea 200 que le pasamos la funcion, para proceder a obtener la información.

In [5]:
def scrapy_nota(url):
    try:
        req = requests.get(url)
    except Exception as e:
        print(f'error: {e}')
        print(f'url: {url}')
        return None
    if req.status_code != 200:
        print(f'eror en status, hubo un error de {req.status_code}')
        return None
    s_nota = BeautifulSoup(req.text,'lxml')
    diccionario = obtener_info(s_nota)
    diccionario['Url'] = url
    return diccionario

Funcion para obtener links de cada sección

In [6]:
def links(soup):
    list_links= list()
    cont = soup.find('div',attrs={'class':'featured-article__container'})
    link_container = cont.a.get('href')
    web_ul = soup.find('ul',attrs={'class':'article-list'})
    list_links.append(link_container)
    for link in web_ul.find_all('h2'):
        list_links.append(link.a.get('href'))
    return list_links

Aplicando nuestra funcion links hemos encontrado los links de las secciones.

In [7]:
url =  'https://www.pagina12.com.ar/secciones/el-pais'
ps2 = requests.get(url)
soup = BeautifulSoup(ps2.text, 'lxml')
links_list = links(soup)

Inicio del Scraper

In [9]:
data = []
for i, nota in enumerate(links_list):
    print(f'Scrapeando nota{i}/{len(links_list)}')
    data.append(scrapy_nota(nota))

Scrapeando nota0/15
Scrapeando nota1/15
Scrapeando nota2/15
Scrapeando nota3/15
Scrapeando nota4/15
Scrapeando nota5/15
Scrapeando nota6/15
Scrapeando nota7/15
Scrapeando nota8/15
Scrapeando nota9/15
Scrapeando nota10/15
Scrapeando nota11/15
Scrapeando nota12/15
Scrapeando nota13/15
Scrapeando nota14/15


estructurando la informacion a un Data Frame con la libreria de pandas

In [10]:
df = pd.DataFrame(data)

In [12]:
df

,Fecha,Titulo,Volanda,Copete,Autor,Imagen,Cuerpo de texto,Url
0,2020-08-24,Paridad de género: Sigue la puja por la banca ...,Debate por la intepretación de la ley,Tras la designación de Darío Martínez en la se...,Por Julián Bruschtein,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,La designación del diputado del Frente de Todo...,https://www.pagina12.com.ar/287287-paridad-de-...
1,2020-08-24,"Cristina Castro: ""Si queremos vivir mejor, que...",Tras el encuentro con el Presidente habló la m...,Durante una conferencia convocada junto con la...,Por Julián Bruschtein,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,“No podemos bajar los brazos. Si queremos vivi...,https://www.pagina12.com.ar/287243-cristina-ca...
2,2020-08-24,"Quién es Santiago Ulpiano Martínez, el fiscal ...",Su nombramiento tuvo numerosas impugnaciones d...,La madre del joven pide que sea apartado de la...,Por Ailín Bullentini,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,Hace un mes que Cristina Castro recusó por pri...,https://www.pagina12.com.ar/287237-quien-es-sa...
3,2020-08-24,La reunión entre Alberto Fernández y la madre ...,Cristina Castro valoró el encuentro con el man...,None,Por Julián Bruschtein,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,El presidente Alberto Fernández mantuvo una re...,https://www.pagina12.com.ar/287204-la-reunion-...
4,2020-08-24,Macri reconoció la charla con Alberto Fernánde...,Luego de que el Presidente revelara que le pla...,None,Por Julián Bruschtein,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,Mauricio Macri reconoció hoy la existencia de ...,https://www.pagina12.com.ar/287200-macri-recon...
5,2020-08-24,Coronavirus: se desacelera el crecimiento de c...,"Lo confirmó Daniel Gollan, ministro de Salud b...","""Estamos logrando comenzar a amesetar la curva...",Por Julián Bruschtein,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"""Empieza a haber un descenso marcado de la vel...",https://www.pagina12.com.ar/287192-coronavirus...
6,2020-08-24,Río Negro: vuelta a fase 1 y protestas contra ...,El departamento de General Roca entró en aisla...,None,Por Julián Bruschtein,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,El gobierno de Río Negro anunció que el depart...,https://www.pagina12.com.ar/287172-rio-negro-v...
7,2020-08-24,Alberto Fernández ratificó su frase contra Mac...,Los tuits del Presidente en la madrugada,Luego de recibir críticas de la oposición por ...,Por Julián Bruschtein,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"El presidente Alberto Fernández compartió, a t...",https://www.pagina12.com.ar/287166-alberto-fer...
8,2020-08-24,Denuncian un ataque intimidatorio a la sede de...,Dejaron un cuchillo en el escritorio de la del...,None,Por Julián Bruschtein,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,La sede del Instituto Nacional contra la Discr...,https://www.pagina12.com.ar/287164-denuncian-u...
9,2020-08-24,"Reforma judicial: qué piensan juezas, fiscales...",Elogios a dos artículos del texto y críticas s...,La titular de la Unidad Fiscal Especializada d...,Por Julián Bruschtein,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,¿Tiene perspectiva de género el proyecto de re...,https://www.pagina12.com.ar/287098-reforma-jud...


exportando el Data Frame a un documento CSV

In [ ]:
df.to_csv('Pagina12.csv')